# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver. Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Python also allows executing SQL queries and getting the result into a Python object, for example a Pandas data frame. Instead of exporting a .csv file from DBeaver you can directly get the data you need into Python and continue your work. In addition we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

After you have the data in Python in the required shape you can export the data into a .csv file. This file is for your own reference, please avoid sending .csv files around - database is the point of reference when it comes to data. 

Having a copy of a .csv file (or another format) can speed up your analysis work. Imagine that the query takes 25 minutes to run. If you made some mistakes in your Python code you might need to go back to the original dataset. Instead of having to rerun the SQL query and having to wait you can read in the .csv file you have previously saved on your hard disk into Python and continue with your analysis work. 

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

My Client is Nicole Johnson.(Buyer)
His requirements: the house should be
1. lively
2. central neighborhood
3. middle price range
4. right timing(within a year)
I'm working on The King County House Prices dataset which includes data from May 2014 to May 2015 from the state of Washington DC. The King County House Sales dataset on contains 21,597 rows and 21 columns.

Business Questions:
1. How is location affecting the prices in King County and how should you price your house based on the location?
2. What is the factor that is not affecting the price of a house?
3. What is the main factor outside the square foot that's driving the price
4. Does the age of a house affect its sale price in central and lively neighborhoods, for houses in the mid-price range?
5. Is there any pick time when the price drop within the year? when and how could it benefits my client?



### Connecting via psycopg2

In [ ]:
import pandas as pd
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px


In [ ]:
# Floats (decimal numbers) should be displayed rounded with 2 decimal places
pd.options.display.float_format = "{:,.2f}".format
# Set style for plots
plt.style.use('fivethirtyeight')


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a `.env` file which is added into the `.gitignore`. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
(We will share the file with you on Slack!)


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

The function from the psycopg2 package to create a connection is called `connect()`.
`connect()` expects the parameters listed above as input in order to connect to the database.

In [ ]:
# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

### Retrieving data from the database with psycopg2

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [ ]:
cur = conn.cursor()

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [ ]:
df = pd.read_csv('data/_SELECT_FROM_king_county_house_details_kchd_FULL_JOIN_king_count_202305111223.csv')
#cur.execute("SELECT * FROM eda.king_county_house_sales LIMIT 10")
#cur.fetchall()
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
df.describe()

In [ ]:
df.info()


In [ ]:
# Plot a scatter plot of longitude and latitude against price
plt.scatter(df['long'], df['lat'], c=df['price'], alpha=0.5)
plt.colorbar().set_label('Price')

# Add labels and title
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('House Prices by Location')

# Show the plot
plt.show()

From the above scatter plot, we can observe that there is a positive correlation between the price of the house and its location. This means that as we move closer to the central area, the prices tend to increase. However, we can also see that there are certain locations that have high prices despite being farther away from the central area. This could be due to factors such as the neighborhood, amenities, or view. Overall, the scatter plot confirms our hypothesis that the location of a house affects its price.

In [ ]:
# plot the map to show the mid price range in the central area

# Filter the data for mid-price range and central area
filtered_data = df[(df['price'] > 300000) & (df['price'] < 600000) & 
                     (df['lat'] > 47.5) & (df['lat'] < 47.8) &
                     (df['long'] > -122.4) & (df['long'] < -122.2)]

# Create the geographic visualization
fig = px.scatter_mapbox(filtered_data, lat='lat', lon='long', color='price',
                        size='sqft_living', hover_name='zipcode', zoom=10,
                        mapbox_style='open-street-map')

# Update the layout of the visualization to include a title and remove margins
fig.update_layout(title='Mid-Price Range Houses in Central Area', margin=dict(l=0, r=0, t=50, b=0))

# Add interactivity with hover information
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>' +
                                  'Price: %{marker.color:.2f}<br>' +
                                  'Square Footage: %{marker.size}<br>' +
                                  'Latitude: %{lat:.2f}<br>' +
                                  'Longitude: %{lon:.2f}')

# Show the visualization
fig.show()

This is a scatter mapbox plot showing the locations of houses in the mid-price range in the central area of King County. The color of each dot represents the sale price, and the size represents the square footage of the living area

## Insights and recommendation

Zip code 98103 corresponds to the area around the neighborhood of Fremont, which is located in the north-central part of Seattle.
Zip code 98115 corresponds to the area around the neighborhoods of Wedgwood and View Ridge, which are located in the northeast part of Seattle.
Zip code 98117 corresponds to the area around the neighborhood of Ballard, which is located in the northwest part of Seattle.

In [ ]:
# Create a scatterplot with the longitude and latitude as the x and y axes, and use the price as the color
# sns.scatterplot(x='long', y='lat', hue='price', data=df, size= 'price',
#                 sizes=(20, 200), alpha=0.7, legend=True)

# # Add a title to the plot
# plt.title('King County House Prices by Location')

# # Show the plot
# plt.show()

In [ ]:
df.shape

Cleaning the dataframe

In [ ]:
# change "date" dtype to datetime with format %Y/%m/%d
df1 = df.copy()
df1['date'] = pd.to_datetime(df1['date'], format='%Y/%m/%d')
df1


In [ ]:
#checking the dtypes
df1.info()

In [ ]:
#column yr_renovated has a lot of zero
df2 = df1.copy()
df2['yr_renovated'] = df2.yr_renovated.apply(lambda x: (x /10))

#also changing the data type

df2['yr_renovated'] = df2['yr_renovated'].astype('Int64')
df2.head()

In [ ]:
df2.isnull().sum()

## Correlation matrix

In [ ]:
#correlation matrix between columns
Entire_dataframe = df2[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'date'
       ]]
#heatmap to see the correlation
df_matrix = Entire_dataframe.corr()
plt.figure(figsize = (20, 14))
ax = sns.heatmap(df_matrix, linewidths=.4, annot=True, cmap='coolwarm')

In [ ]:
#fill missing values

df3 = df2.copy()
df3['view'] = df2['view'].fillna(df2['view'].mean())
df3['yr_renovated'] = df2['yr_renovated'].fillna(df2['yr_renovated'].mode()[0])
df3.head()


In [ ]:
# Convert date to datetime format and extract year and month
df3['date'] = pd.to_datetime(df3['date'])
df3['year_month'] = df3['date'].dt.to_period('M')

# Group by year and month, and calculate the median price
median_price_by_month = df3.groupby('year_month')['price'].median().reset_index()

# Compute the price per sqft for each variable
df3['price_sqft_living'] = df3['price'] / df3['sqft_living']
df3['price_sqft_lot'] = df3['price'] / df3['sqft_lot']
df3['price_sqft_lot15'] = df3['price'] / df3['sqft_lot15']
df3['price_sqft_living15'] = df3['price'] / df3['sqft_living15']

# Calculate the average price per sqft
df3['price_sqft_avg'] = (df3['price_sqft_living'] + df3['price_sqft_lot'] + df3['price_sqft_lot15'] + df3['price_sqft_living15']) / 4

# Group by year and month, and calculate the mean price per sqft
mean_price_sqft_by_month = df3.groupby('year_month')['price_sqft_avg'].mean().reset_index()
df3.head()

In [ ]:
# plt.figure(figsize = (12, 6))

# sns.countplot(x = df3['view'], hue = 'price', data = df3)

plt.figure(figsize=(12, 6))
df3['view'].value_counts().plot(kind='bar')
plt.xlabel('View')
plt.ylabel('Count')
plt.title('Distribution of House Sales by View')
plt.show()


# this shows that view does not impact the sales of a house much

In [ ]:
plt.figure(figsize = (18, 8))
sns.barplot(x = df3['view'], y = df3['price'], hue = df3['price_sqft_avg'], ci = False)
plt.title('Average Price based on view per price group')

# There's is no major significance of average price and view if we compared it per price group
# the analogy here is even if your houses has the most beautiful view in King County it shouldn't affect the price much

## Need to work on, it doesn't work

In [ ]:
# plt.figure(figsize = (12 , 5))
# sns.barplot(x = df3['price_sqft_avg'], y= df3['yr_built'], ci = False, palette = 'icefire_r')
# plt.title('Average Age per Price Group')
# plt.xticks(rotation = 90)
# plt.show()
# the age group of the house doesn't really have a positve or negative correlation with the price
# house that's on the lower price tend to be older in average

## need some modification

In [ ]:
# Convert date column to datetime format
df3['date'] = pd.to_datetime(df3['date'])

# Compute median sale price and average price per sqft by month
df3['month'] = pd.to_datetime(df3['date']).dt.to_period('M')
median_price_by_month = df3.groupby('month')['price'].median()
avg_price_by_month = df3.groupby('month')['price_sqft_avg'].mean()

# Create a figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharex=True)

# Plot median sale price over time
axs[0].plot(median_price_by_month.index.to_timestamp(), median_price_by_month.values)
axs[0].set_ylabel('Median Sale Price')
axs[0].set_title('Median Sale Price Over Time')

# Plot average price per sqft over time
axs[1].plot(avg_price_by_month.index.to_timestamp(), avg_price_by_month.values)
axs[1].set_ylabel('Avg Price per Sqft')
axs[1].set_title('Avg Price per Sqft Over Time')

plt.show()


There is a pattern to both graphs. Prices dip some time around January and February 2015. I proceeded to split the data into two data sets. Homes sold up to January 2015, and homes sold from February 2015 on. is this true for the above graph

this solves my question: Is there any pick time when the price drop within the year? when and how could it benefits my client?

In [ ]:
#add column total sqft which is the sum of sqft_living and sqft_lot

df3['total_sqft'] = df3.apply(lambda x: x['sqft_living'] + x['sqft_lot'], axis=1)

In [ ]:
# calculate price per square foot of total_sqft space

#df.price_sqft = df4.copy()

df3['price_sqft'] = df3.apply(lambda row: row['price'] / row['total_sqft'], axis=1)
df3.head()

In [ ]:
#add a new column distance to get the mean value of lat and long

df3['distance'] = df3.apply(lambda x: (x['lat'] + x['long'])/2, axis=1)
df3.head()


In [ ]:
df3.corr()['price_sqft'].sort_values()

In [ ]:
df4 = df3.copy()
df4 = df3[['condition', 'grade', 'yr_built', 'date', 'distance', 'total_sqft', 'price_sqft']]
df4.head()

In [ ]:
df5.describe()

## Check for correlations

In [ ]:
#Example of choosing which columns to correlate matrix between these variables
#Just a list of the columns of interest inside an extra pair of [ ]
df_matrix1 = df4[["bedrooms","bathrooms","floors","zipcode","condition","grade","yr_built","yr_renovated", "lat", "long", "price","sqft_living","sqft_lot","sqft_living15","sqft_lot15","sqft_above","sqft_basement","price_sqft_living", "distance"]].corr()
#sns.heatmap(df_matrix1)

In [ ]:
# draw the heatmap 
plt.figure(figsize = (20,18))
ax = sns.heatmap(df_matrix1, linewidths=.5, annot=True, cmap='coolwarm')
